# Group 10: The Potential of Professional Tennis Players: A Predictive Model for Player's Best Rank.

### Background information:
    The ATP (Association of Tennis Professionals) Tour is a worldwide tennis tour for men’s professional tennis organized annually by the ATP since 1990. The tour is divided into several tiers of tournaments, with higher tiers offering more ranking points. The PIF ATP Rankings is a merit-based method used by the ATP to rank players, determine entry qualification, and organize tournament seeding. Points are awarded based on the stage of the tournament reached and the tier. The rankings are updated weekly and rewarded points are dropped 52 weeks after being awarded, with some exceptions.


### Our research question:
    We plan to predict the best ranking of players based on various predictors. What is a player’s predicted best rank?

### The dataset:
    We will be using a data set containing player stats for the top 500 ATP players from 2017-2019. The dataset contains 37 variables, including both quantitative and qualitative values. 




## Preliminary Exploratory Data Analysis

In [1]:
# Retrieving nessesary packages
library(dplyr)
library(tidyverse)
library(rvest)
library(tidymodels)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.4     ✔ stringr   1.5.0
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding


── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.

In [33]:
# Reading the data file
url = "https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS"
player_data = read_csv(url)

# Cleaning the data and getting what we need: Best Rank, 
processed_player_data = player_data |>
                        separate(Age, c("Age", "Birth date"), sep = " ") |>
                        mutate(Age = as.numeric(Age)) |>
                        separate(`Current Rank`, c("Current Rank", "Current Rank (Other)"), sep = " ") |>
                        mutate(`Current Rank` = as.numeric(`Current Rank`)) |>

                        separate(`Best Rank`, c("Best Rank", "Best Rank (Date)"), sep = " ") |>
                        mutate(`Best Rank` = as.numeric(`Best Rank`)) |>

                        separate(`Height`, c("Height (cm)", "cm REMOVE"), sep = " ") |>
                        mutate(`Height (cm)` = as.numeric(`Height (cm)`)) |>
                        separate(`Weight`, c("Weight (kg)", "kg REMOVE"), sep = " ") |>
                        mutate(`Weight (kg)` = as.numeric(`Weight (kg)`)) |>
                        mutate(Country = as.factor(Country)) |>
                        mutate(Backhand = as.factor(Backhand)) |>
                        mutate(Plays = as.factor(Plays)) |>
                        select(Age, Country, Plays, `Current Rank`, `Best Rank`, Backhand, `Height (cm)`, `Turned Pro`, Seasons, Titles, `Weight (kg)`)
                        
                    
processed_player_data

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Age,Country,Plays,Current Rank,Best Rank,Backhand,Height (cm),Turned Pro,Seasons,Titles,Weight (kg)
<dbl>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
26,Brazil,Right-handed,378,363,NA,NA,NA,NA,NA,NA
18,United Kingdom,Left-handed,326,316,Two-handed,NA,NA,NA,NA,NA
32,Slovakia,Right-handed,178,44,Two-handed,185,2005,14,NA,NA
21,"Korea, Republic of",Right-handed,236,130,Two-handed,NA,NA,2,NA,NA
27,Australia,Right-handed,183,17,Two-handed,193,2008,11,4,NA
22,Poland,Right-handed,31,31,Two-handed,NA,2015,5,1,NA
28,United States,Right-handed,307,213,Two-handed,NA,2010,1,NA,NA
21,"Taiwan, Province of China",Right-handed,232,229,Two-handed,NA,NA,1,NA,NA
25,Uzbekistan,Right-handed,417,253,Two-handed,NA,NA,5,NA,NA


In [ ]:
# Summary for data
data_summary = data

data_summary

In [ ]:
# Visualization for data
options(repr.plot.width=20, repr.plot.height=20)

data_plot = data |>
            ggplot(aes(x=..,y=..)) +
            geom_point() +
            xlab("x") +
            ylab("y") +
            ggTitle("x vs y") +
            theme(text = element_text(size=20))

data_plot

## Methods

### Relevant variables:
Predicted Variable: Best Rank
  
Predictors:
- Age
- Current Rank
- Height
- Year Turned pro
- Current ELO Rank
- Best ELO Rank
- Peak ELO Rating
- Weight
- Plays (Left-handed or Right-handed)
- Backhand (Two-Handed or One-Handed)
- Seasons Played
- Country


### Outlined procedure:
1. one
2. two
3. three
4. four
5. five

### Visualization:
We plan to graph a scatter + line plot to visualize our regression model.


## Expected Outcomes and Significance

### Expected findings:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam in sagittis velit. Pellentesque eget pretium ipsum. Proin sed dui neque. Nulla venenatis enim sit amet ipsum consequat, placerat euismod odio mattis. Nunc sed lobortis magna, sed ultricies justo. Etiam aliquet dictum metus non lobortis. Aenean sed suscipit arcu. Sed nunc tellus, condimentum id facilisis elementum, egestas ut libero. Donec sit amet elementum enim, eu luctus mi.v

### Relevancy of findings:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam in sagittis velit. Pellentesque eget pretium ipsum. Proin sed dui neque. Nulla venenatis enim sit amet ipsum consequat, placerat euismod odio mattis. Nunc sed lobortis magna, sed ultricies justo. Etiam aliquet dictum metus non lobortis. Aenean sed suscipit arcu. Sed nunc tellus, condimentum id facilisis elementum, egestas ut libero. Donec sit amet elementum enim, eu luctus mi.v

### Future studies:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam in sagittis velit. Pellentesque eget pretium ipsum. Proin sed dui neque. Nulla venenatis enim sit amet ipsum consequat, placerat euismod odio mattis. Nunc sed lobortis magna, sed ultricies justo. Etiam aliquet dictum metus non lobortis. Aenean sed suscipit arcu. Sed nunc tellus, condimentum id facilisis elementum, egestas ut libero. Donec sit amet elementum enim, eu luctus mi.v